In [12]:
import torchaudio

# Example: list of file paths
file_paths = ["/home/sonlt/workspace/markdocdown/EGGM/data/cremad/AudioWAV/1001_DFA_ANG_XX.wav", 
              "/home/sonlt/workspace/markdocdown/EGGM/data/cremad/AudioWAV/1001_DFA_FEA_XX.wav", 
              "/home/sonlt/workspace/markdocdown/EGGM/data/cremad/AudioWAV/1001_IEO_DIS_MD.wav"]

# Load and preprocess audio files (mono, 16kHz)
waveforms = []

for path in file_paths:
    waveform, sr = torchaudio.load(path)
    waveform = torchaudio.functional.resample(waveform, orig_freq=sr, new_freq=16000)
    waveform = waveform.mean(dim=0)  # mono
    waveforms.append(waveform)

waveforms_np = [w.numpy() for w in waveforms]

In [24]:
from transformers import ASTFeatureExtractor, ASTForAudioClassification

feature_extractor = ASTFeatureExtractor.from_pretrained("MIT/ast-finetuned-audioset-10-10-0.4593")

# Let the feature extractor handle variable-length padding
inputs = feature_extractor(
    waveforms_np,
    sampling_rate=16000,
    return_tensors="pt",
    pad=True
)
inputs.input_values.shape

torch.Size([3, 1024, 128])

In [33]:
model = ASTForAudioClassification.from_pretrained("MIT/ast-finetuned-audioset-10-10-0.4593")

outputs = model(inputs.input_values, output_hidden_states=True)
outputs.keys()


odict_keys(['logits', 'hidden_states'])

In [42]:
model.audio_spectrogram_transformer(inputs.input_values).pooler_output.shape

torch.Size([3, 768])

In [37]:
outputs.hidden_states[-1].shape

torch.Size([3, 1214, 768])